In [1]:
%%capture
%run 02_transaction.ipynb
%run 03_merkle.ipynb
%run 04_account.ipynb 

In [2]:
from objsize import get_deep_size as get_size

## Block Header

The `Header` is a blocks summary. It only stores the `root` node of the txs merkle tree, which makes it small and constant in size. The actual mining happens on the block header.

In [3]:
class Header(Hashable):
    def __init__(self, root, prev_hash, number, n_txs):
        self.root      = root
        self.prev_hash = prev_hash
        self.number    = number
        self.n_txs     = n_txs
        self.mined     = False
        self.time      = time.time()
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val) 
        if prop not in ['nonce', 'hash']:
            super().__setattr__('hash', sha(self.__dict__))
            
    def __str__(self):  return stringfy(self)

In [4]:
bh = Header(rh(),rh(),2,10); print(bh)

root:          📀 0x5a270ae7cc...de1
hash:          🔎 0xa882c73a2b...4cb
prev_hash:     📅 0x5fd4340c76...b49
number:        2
n_txs:         10
mined:         False
time:          1617485003.102099 eth


If anything in the block header changes its `hash` changes automatically as well.

In [5]:
bh_changed        = deepcopy(tx2)
bh_changed.number = 120
assert bh != bh_changed

### Info

Includes all extra informations that are irrelevant for mining and therefore are not in the block header.

Calculate the total txs `volume` and `fees` included in a block.

In [6]:
def stats(txs):
    volume = sum([tx.value for tx in txs])
    fees   = sum([tx.fee   for tx in txs])
    return volume, fees

`Info` calculates some interesting facts about the `txs` that will be included in the block.

In [7]:
class Info:
    def __init__(self, txs): self.volume,self.fees = stats(txs)
    def __str__(self): return stringfy(self)

## Block

Consists of the block header `bh` and a list of txs. To validate that the txs are correct, the merkle tree can be rebuild and checked against the root hash stored in the `bh`.

In [8]:
class Block(Hashable): 
    def __init__(self, bh, txs):
        self.info   = Info(txs)
        self.header = bh
        self.hash   = bh.hash
        self.mt     = MerkleTree(txs)
        self.txs    = self.val_txs(txs)
        
    def val_txs(self, txs):
        for tx in txs: assert val_sig(tx),       'tx signature invalid'
        assert self.are_unique(txs),             'txs include duplicates'
        assert self.mt.root == self.header.root, 'txs root hash do not match'
        return {tx.hash: tx for tx in txs}
    
    def json(self): 
        info = self.info.__dict__
        h    = self.header.__dict__
        txs  = [tx.json() for tx in self.txs.values()]
        d    = {**info, **h, 'txs': txs}
        #d['time'] = time.ctime(d['time'])
        return json.dumps(d, indent=4)
    
    def are_unique(self, txs): return len(set([tx.hash for tx in txs])) == len(txs)
    def __getitem__(self, key):return self.txs[key] 
    def __str__(self):         return (str(self.header)
                                       +'\n'+str(self.info)
                                       +'\n\ntxs:\n' 
                                       +'\n'.join(tx.smry() for tx in self.txs.values()))

Create random list of signed txs.

In [9]:
acc1,acc2 = Account(),Account()
def r_stxs(n): return [acc1.signed_tx(acc2,ri(1,9),ri(1,9)/100) for _ in range(n)]

Create block from a block header containing its txs merkle tree.

In [10]:
txs = r_stxs(4)
mt  = MerkleTree(txs)
bh  = Header(mt.root, rh(), 2, len(txs))
b   = Block(bh, txs); print(b)

root:          🕁 0xdbbfe5f838...d07
hash:          💖 0x304e59744b...7cc
prev_hash:     💝 0x37057276e8...2fc
number:        2
n_txs:         4
mined:         False
time:          1617485003.144881 eth
volume:        16.0 eth
fees:          0.31 eth

txs:
🔜 0xB65F -> 📂 0x5cAe 6.0 eth
🔜 0xB65F -> 📂 0x5cAe 1.0 eth
🔜 0xB65F -> 📂 0x5cAe 2.0 eth
🔜 0xB65F -> 📂 0x5cAe 7.0 eth


### Block JSON

Every block has a json representation that we will use for our API later.

In [11]:
block_json = b.json()

We can load this JSON string `d` and create a block object again.

In [12]:
def load_block(d):
    d  = json.loads(d)
    txs = []
    for tx in d['txs']: txs.append(load_tx(tx))
    bh = Header(d['root'],d['prev_hash'],d['number'],d['n_txs']) 
    for k,v in d.items(): setattr(bh, k, v)
    bh.hash = d['hash']
    b = Block(bh, txs)
    return b
    
b_from_json = load_block(b.json())

In [13]:
assert b == b_from_json

#### Access specific tx from block with its hash.

In [15]:
tx_hash = next(iter(b.txs.keys())); tx_hash

'0x2779ea26e3ae8d58b5ef792d2e8d200dfee12e44e3d8fecc0419738b7cb0bc95'

In [16]:
tx = b[tx_hash]; print(tx)

fr:            🔜 0xB65F8c217c...dc6
hash:          💍 0x2779ea26e3...c95
to:            📂 0x5cAe54987C...6Dd
value:         6.0 eth
fee:           0.09 eth
nonce:         0
time:          1617485003.12468 eth
signed:        True
sig:           💲 0x4c36a6eb87...61c


Blocks can only contain unique txs. 

In [17]:
txs[0] = txs[1]

mt  = MerkleTree(txs)
bh  = Header(mt.root, rh(), 2, len(txs))

In [18]:
dup_detected = False
try:    Block(bh, txs)
except: dup_detected = True
assert  dup_detected